In [2]:
import pandas as pd
import numpy as np

In [4]:
m = 10000
f = 150

df = pd.read_excel('input2.xlsx', engine='openpyxl', dtype=float)
data = df.to_numpy()

point_left = data[:, 0:2]
point_right = data[:, 2:4]

In [5]:
from spatial_rear_intersection import rotationMatrix
X = np.zeros(5)

R = rotationMatrix(X[2], X[3], X[4])
auxiliary_left = np.hstack((point_left, -150*np.ones((9,1))))
auxiliary_right = (R @ np.hstack((point_right, -150*np.ones((9,1)))).T).T

print(auxiliary_left)
bu = np.mean(point_left[:,0]- point_right[:,0])
bw = bu*X[1]

N_1 = (bu*auxiliary_right[:,2] - bw*auxiliary_right[:,0])/(auxiliary_left[:,0]*auxiliary_right[:,2]-auxiliary_left[:,2]*auxiliary_right[:,1])
N_2 = (bu*auxiliary_left[:,2] - bw*auxiliary_left[:,0])/(auxiliary_left[:,0]*auxiliary_right[:,2]-auxiliary_left[:,2]*auxiliary_right[:,1])

[[  16.012   79.963 -150.   ]
 [  88.56    81.134 -150.   ]
 [  13.362  -79.37  -150.   ]
 [  82.24   -80.027 -150.   ]
 [  51.758   80.555 -150.   ]
 [  14.618   -0.231 -150.   ]
 [  49.88    -0.782 -150.   ]
 [  86.14    -1.346 -150.   ]
 [  48.035  -79.962 -150.   ]]


In [35]:
a = - auxiliary_right[:,0] * auxiliary_right[:,1] * N_2 /auxiliary_right[:,2]

b = -(auxiliary_right[:,2]+auxiliary_right[:,0]*auxiliary_right[:,0]/auxiliary_right[:,2]) * N_2

c = auxiliary_right[:,0] * N_2

d = bu * np.ones(9)

e = - bu * auxiliary_right[:,0] / auxiliary_left[:,2] 

l = N_1 * auxiliary_left[:,1] - N_2 * auxiliary_right[:,1] - bu*X[0]

A = np.vstack((a, b, c, d, e)).T

print(A)


[[ 5.70001127e+01 -2.73950442e+02  1.08632339e+02  9.21222222e+01
  -4.54039726e+01]
 [-2.43044494e+01  1.33339184e+03 -4.66293284e+01  9.21222222e+01
  -3.22550607e+00]
 [ 4.15535179e+01  1.91488377e+02 -7.90201154e+01  9.21222222e+01
  -4.85926298e+01]
 [ 2.99581119e+00  8.54953090e+01 -5.61090385e+00  9.21222222e+01
  -6.07208274e+00]
 [ 7.20932939e+01 -5.54153250e+02  1.37822848e+02  9.21222222e+01
  -2.45370610e+01]
 [-1.15240844e-01  1.19093493e+03 -4.80170184e+02  9.21222222e+01
  -4.66789441e+01]
 [ 5.20369625e-01  2.92956755e+02 -7.63751896e+01  9.21222222e+01
  -2.59176660e+01]
 [ 1.13258683e-01  1.56991376e+02 -8.04394058e+00  9.21222222e+01
  -4.73262563e+00]
 [ 1.70313786e+01  1.17641043e+02 -3.20395654e+01  9.21222222e+01
  -2.72915154e+01]]
